# Filght Homework

## Install package

In [ ]:
install.packages("nycflights13")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(tidyverse)
library(nycflights13)
library(glue)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.7      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## View Data

In [ ]:
glimpse(flights)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

In [ ]:
apply(flights, MARGIN = 2, function(col) sum(is.na(col)))

year          month            day       dep_time sched_dep_time 
             0              0              0           8255              0 
     dep_delay       arr_time sched_arr_time      arr_delay        carrier 
          8255           8713              0           9430              0 
        flight        tailnum         origin           dest       air_time 
             0           2512              0              0           9430 
      distance           hour         minute      time_hour 
             0              0              0              0

The missing values are seen in time-related columns. It's better to drop rows than imputation.

In [ ]:
df_flight <- tibble(drop_na(flights))
apply(df_flight, MARGIN = 2, function(col) sum(is.na(col)))


year          month            day       dep_time sched_dep_time 
             0              0              0              0              0 
     dep_delay       arr_time sched_arr_time      arr_delay        carrier 
             0              0              0              0              0 
        flight        tailnum         origin           dest       air_time 
             0              0              0              0              0 
      distance           hour         minute      time_hour 
             0              0              0              0

In [ ]:
glimpse(airlines)

Rows: 16
Columns: 2
$ carrier <chr> "9E", "AA", "AS", "B6", "DL", "EV", "F9", "FL", "HA", "MQ", "O…
$ name    <chr> "Endeavor Air Inc.", "American Airlines Inc.", "Alaska Airline…


### Q1:Which carrier had the most filghts on May 30, 2013?

In [ ]:
df1 <- df_flight %>%
  filter(day == 30, month == 5, year == 2013) %>%
  count(carrier) %>%
  arrange(desc(n)) %>%
  left_join(airlines, by="carrier") %>%
  head(5)
print(df1)

# A tibble: 5 × 3
  carrier     n name                    
  <chr>   <int> <chr>                   
1 UA        173 United Air Lines Inc.   
2 EV        170 ExpressJet Airlines Inc.
3 B6        154 JetBlue Airways         
4 DL        136 Delta Air Lines Inc.    
5 AA         89 American Airlines Inc.  


### Q2: Which carrier had the most delay of departure?

In [ ]:
df2 <- df_flight %>%
  select(carrier, dep_delay) %>%
  filter(dep_delay > 0) %>%
  group_by(carrier) %>%
  summarise(
    dept_delay = sum(dep_delay > 0)
  ) %>%
  arrange(desc(dept_delay)) %>% 
  left_join(airlines, by = "carrier") %>%
  head(5)
print(df2)

# A tibble: 5 × 3
  carrier dept_delay name                    
  <chr>        <int> <chr>                   
1 UA           27125 United Air Lines Inc.   
2 EV           22976 ExpressJet Airlines Inc.
3 B6           21372 JetBlue Airways         
4 DL           15186 Delta Air Lines Inc.    
5 AA           10105 American Airlines Inc.  


### Q3: Which airport (origin) has the most flights to the "IAH" destination?

In [ ]:
glimpse(airports)

Rows: 1,458
Columns: 8
$ faa   <chr> "04G", "06A", "06C", "06N", "09J", "0A9", "0G6", "0G7", "0P2", "…
$ name  <chr> "Lansdowne Airport", "Moton Field Municipal Airport", "Schaumbur…
$ lat   <dbl> 41.13047, 32.46057, 41.98934, 41.43191, 31.07447, 36.37122, 41.4…
$ lon   <dbl> -80.61958, -85.68003, -88.10124, -74.39156, -81.42778, -82.17342…
$ alt   <dbl> 1044, 264, 801, 523, 11, 1593, 730, 492, 1000, 108, 409, 875, 10…
$ tz    <dbl> -5, -6, -6, -5, -5, -5, -5, -5, -5, -8, -5, -6, -5, -5, -5, -5, …
$ dst   <chr> "A", "A", "A", "A", "A", "A", "A", "A", "U", "A", "A", "U", "A",…
$ tzone <chr> "America/New_York", "America/Chicago", "America/Chicago", "Ameri…


In [ ]:
df3 <- df_flight %>%
  select(origin, dest) %>%
  filter(dest == "IAH") %>%
  count(origin) %>%
  arrange(desc(n)) %>%
  left_join(airports,by=c("origin" = "faa")) %>%
  select(origin, n, name) %>%
  head(5)
print(df3)

# A tibble: 3 × 3
  origin     n name               
  <chr>  <int> <chr>              
1 EWR     3923 Newark Liberty Intl
2 LGA     2891 La Guardia         
3 JFK      271 John F Kennedy Intl


### Q4: How far the average distances of each carrier? show top 5.

In [ ]:
df4 <- df_flight %>%
  select(carrier, distance) %>%
  group_by(carrier) %>%
  summarize(avg_dist = mean(distance)) %>%
  arrange(desc(avg_dist)) %>%
  left_join(airlines, by="carrier") %>%
  head(5)
print(df4)

# A tibble: 5 × 3
  carrier avg_dist name                  
  <chr>      <dbl> <chr>                 
1 HA         4983  Hawaiian Airlines Inc.
2 VX         2499. Virgin America        
3 AS         2402  Alaska Airlines Inc.  
4 F9         1620  Frontier Airlines Inc.
5 UA         1531. United Air Lines Inc. 


### Q5: Which carrier had the longest distance on Febuary?

In [ ]:
df5 <- df_flight %>%
  select(carrier, distance, month) %>%
  filter(month == 2) %>%
  group_by(carrier) %>%
  summarize(max_dist = max(distance)) %>%
  arrange(desc(max_dist)) %>%
  left_join(airlines, by="carrier") %>%
  head(5)
print(df5)

# A tibble: 5 × 3
  carrier max_dist name                  
  <chr>      <dbl> <chr>                 
1 HA          4983 Hawaiian Airlines Inc.
2 UA          4963 United Air Lines Inc. 
3 AA          2586 American Airlines Inc.
4 B6          2586 JetBlue Airways       
5 DL          2586 Delta Air Lines Inc.  
